In [1]:
#https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
import folium          
from   folium.plugins import MarkerCluster
import geocoder 

import os
from subprocess import check_output
from branca.element import Figure

# Reading/Copying Csv

In [3]:
df = pd.read_csv('myshiptracking test.csv')
df = df.copy()
df

,Date,Status,To,From,map,Port,Speed
0,2022-03-20 10:37,ETA Changed,3/24/2022 10:30,3/20/2022 5:45,"51.94967, 3.98981",NL MAASVLAKTE - ROTTERDAM,0 kn 34
1,2022-03-20 10:30,Destination Changed,BEANR,NLRTM,"51.94974, 3.99019",NL MAASVLAKTE - ROTTERDAM,0.2 kn 36
2,2022-03-20 10:25,Status Changed,Moored,Constrained by her draught,"51.94987, 3.99076",NL MAASVLAKTE - ROTTERDAM,0.5 kn 40
3,2022-03-20 10:22,STOP Moving,NL,NaN,"51.94982, 3.99148",NL MAASVLAKTE - ROTTERDAM,0.3 kn 41
4,2022-03-20 08:43,PORT ARRIVAL,NL,NaN,"51.96252, 4.07971",NL MAASVLAKTE - ROTTERDAM,4.1 kn 265
5,2022-03-20 03:42,Status Changed,Constrained by her draught,Under way using engine,"51.94307, 3.43059",NLRTM,4.5 kn 88
6,2022-03-20 02:16,ETA Changed,3/20/2022 5:45,3/20/2022 4:45,"51.90014, 3.20315",NLRTM,7.6 kn 65.6
7,2022-03-20 00:29,ETA Changed,3/20/2022 4:45,3/20/2022 3:00,"51.82419, 2.89434",NLRTM,6.2 kn 70.9
8,2022-03-19 23:29,Change Sea Area,Dutch part of the North Sea,Belgian part of the North Sea,"51.77846, 2.70759",NLRTM,8.9 kn 70.4
9,2022-03-19 20:46,Change Sea Area,Belgian part of the North Sea,French part of the North Sea,"51.50706, 2.27365",NLRTM,8.4 kn 43.7


# Columns to lowercase

In [4]:
df.columns= df.columns.str.lower()
df.columns

Index(['date', 'status', 'to', 'from', 'map', 'port', 'speed'], dtype='object')

In [5]:
df.dtypes

date      object
status    object
to        object
from      object
map       object
port      object
speed     object
dtype: object

# Date to Datetime format and break down date into year,month,day,hour,minute

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['hour'] = pd.DatetimeIndex(df['date']).hour
df['minute'] = pd.DatetimeIndex(df['date']).minute

In [8]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible
for row in df['map']:
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # if error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns
df['latitude'] = lat
df['longitude'] = lon

# Resulting Dataframe with Lat and Long, and date breakdown columns

In [9]:
df

,date,status,to,from,map,port,speed,year,month,day,hour,minute,latitude,longitude
0,2022-03-20 10:37:00,ETA Changed,3/24/2022 10:30,3/20/2022 5:45,"51.94967, 3.98981",NL MAASVLAKTE - ROTTERDAM,0 kn 34,2022,3,20,10,37,51.94967,3.98981
1,2022-03-20 10:30:00,Destination Changed,BEANR,NLRTM,"51.94974, 3.99019",NL MAASVLAKTE - ROTTERDAM,0.2 kn 36,2022,3,20,10,30,51.94974,3.99019
2,2022-03-20 10:25:00,Status Changed,Moored,Constrained by her draught,"51.94987, 3.99076",NL MAASVLAKTE - ROTTERDAM,0.5 kn 40,2022,3,20,10,25,51.94987,3.99076
3,2022-03-20 10:22:00,STOP Moving,NL,NaN,"51.94982, 3.99148",NL MAASVLAKTE - ROTTERDAM,0.3 kn 41,2022,3,20,10,22,51.94982,3.99148
4,2022-03-20 08:43:00,PORT ARRIVAL,NL,NaN,"51.96252, 4.07971",NL MAASVLAKTE - ROTTERDAM,4.1 kn 265,2022,3,20,8,43,51.96252,4.07971
5,2022-03-20 03:42:00,Status Changed,Constrained by her draught,Under way using engine,"51.94307, 3.43059",NLRTM,4.5 kn 88,2022,3,20,3,42,51.94307,3.43059
6,2022-03-20 02:16:00,ETA Changed,3/20/2022 5:45,3/20/2022 4:45,"51.90014, 3.20315",NLRTM,7.6 kn 65.6,2022,3,20,2,16,51.90014,3.20315
7,2022-03-20 00:29:00,ETA Changed,3/20/2022 4:45,3/20/2022 3:00,"51.82419, 2.89434",NLRTM,6.2 kn 70.9,2022,3,20,0,29,51.82419,2.89434
8,2022-03-19 23:29:00,Change Sea Area,Dutch part of the North Sea,Belgian part of the North Sea,"51.77846, 2.70759",NLRTM,8.9 kn 70.4,2022,3,19,23,29,51.77846,2.70759
9,2022-03-19 20:46:00,Change Sea Area,Belgian part of the North Sea,French part of the North Sea,"51.50706, 2.27365",NLRTM,8.4 kn 43.7,2022,3,19,20,46,51.50706,2.27365


In [10]:
df = df[['date', 'year', 'month', 'day', 'hour', 'minute','status', 'to', 'from', 'map', 'latitude', 'longitude','port', 'speed']]
df

,date,year,month,day,hour,minute,status,to,from,map,latitude,longitude,port,speed
0,2022-03-20 10:37:00,2022,3,20,10,37,ETA Changed,3/24/2022 10:30,3/20/2022 5:45,"51.94967, 3.98981",51.94967,3.98981,NL MAASVLAKTE - ROTTERDAM,0 kn 34
1,2022-03-20 10:30:00,2022,3,20,10,30,Destination Changed,BEANR,NLRTM,"51.94974, 3.99019",51.94974,3.99019,NL MAASVLAKTE - ROTTERDAM,0.2 kn 36
2,2022-03-20 10:25:00,2022,3,20,10,25,Status Changed,Moored,Constrained by her draught,"51.94987, 3.99076",51.94987,3.99076,NL MAASVLAKTE - ROTTERDAM,0.5 kn 40
3,2022-03-20 10:22:00,2022,3,20,10,22,STOP Moving,NL,NaN,"51.94982, 3.99148",51.94982,3.99148,NL MAASVLAKTE - ROTTERDAM,0.3 kn 41
4,2022-03-20 08:43:00,2022,3,20,8,43,PORT ARRIVAL,NL,NaN,"51.96252, 4.07971",51.96252,4.07971,NL MAASVLAKTE - ROTTERDAM,4.1 kn 265
5,2022-03-20 03:42:00,2022,3,20,3,42,Status Changed,Constrained by her draught,Under way using engine,"51.94307, 3.43059",51.94307,3.43059,NLRTM,4.5 kn 88
6,2022-03-20 02:16:00,2022,3,20,2,16,ETA Changed,3/20/2022 5:45,3/20/2022 4:45,"51.90014, 3.20315",51.90014,3.20315,NLRTM,7.6 kn 65.6
7,2022-03-20 00:29:00,2022,3,20,0,29,ETA Changed,3/20/2022 4:45,3/20/2022 3:00,"51.82419, 2.89434",51.82419,2.89434,NLRTM,6.2 kn 70.9
8,2022-03-19 23:29:00,2022,3,19,23,29,Change Sea Area,Dutch part of the North Sea,Belgian part of the North Sea,"51.77846, 2.70759",51.77846,2.70759,NLRTM,8.9 kn 70.4
9,2022-03-19 20:46:00,2022,3,19,20,46,Change Sea Area,Belgian part of the North Sea,French part of the North Sea,"51.50706, 2.27365",51.50706,2.27365,NLRTM,8.4 kn 43.7


In [11]:
df.drop('map', axis=1, inplace=True)
df

C:\Users\eddie\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,date,year,month,day,hour,minute,status,to,from,latitude,longitude,port,speed
0,2022-03-20 10:37:00,2022,3,20,10,37,ETA Changed,3/24/2022 10:30,3/20/2022 5:45,51.94967,3.98981,NL MAASVLAKTE - ROTTERDAM,0 kn 34
1,2022-03-20 10:30:00,2022,3,20,10,30,Destination Changed,BEANR,NLRTM,51.94974,3.99019,NL MAASVLAKTE - ROTTERDAM,0.2 kn 36
2,2022-03-20 10:25:00,2022,3,20,10,25,Status Changed,Moored,Constrained by her draught,51.94987,3.99076,NL MAASVLAKTE - ROTTERDAM,0.5 kn 40
3,2022-03-20 10:22:00,2022,3,20,10,22,STOP Moving,NL,NaN,51.94982,3.99148,NL MAASVLAKTE - ROTTERDAM,0.3 kn 41
4,2022-03-20 08:43:00,2022,3,20,8,43,PORT ARRIVAL,NL,NaN,51.96252,4.07971,NL MAASVLAKTE - ROTTERDAM,4.1 kn 265
5,2022-03-20 03:42:00,2022,3,20,3,42,Status Changed,Constrained by her draught,Under way using engine,51.94307,3.43059,NLRTM,4.5 kn 88
6,2022-03-20 02:16:00,2022,3,20,2,16,ETA Changed,3/20/2022 5:45,3/20/2022 4:45,51.90014,3.20315,NLRTM,7.6 kn 65.6
7,2022-03-20 00:29:00,2022,3,20,0,29,ETA Changed,3/20/2022 4:45,3/20/2022 3:00,51.82419,2.89434,NLRTM,6.2 kn 70.9
8,2022-03-19 23:29:00,2022,3,19,23,29,Change Sea Area,Dutch part of the North Sea,Belgian part of the North Sea,51.77846,2.70759,NLRTM,8.9 kn 70.4
9,2022-03-19 20:46:00,2022,3,19,20,46,Change Sea Area,Belgian part of the North Sea,French part of the North Sea,51.50706,2.27365,NLRTM,8.4 kn 43.7


# Separating GPS coordinates (Lat and Long) from the rest of the data

In [12]:
#  create results dataframe
df_locations = pd.DataFrame(data = df, columns = {'latitude','longitude'})
df_locations.columns = ['latitude','longitude']  
 

In [13]:
df.dtypes

date         datetime64[ns]
year                  int64
month                 int64
day                   int64
hour                  int64
minute                int64
status               object
to                   object
from                 object
latitude             object
longitude            object
port                 object
speed                object
dtype: object

In [14]:
df['latitude'] = pd.to_numeric(df['latitude'],errors = 'coerce')
df['longitude'] = pd.to_numeric(df['longitude'],errors = 'coerce')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       10 non-null     datetime64[ns]
 1   year       10 non-null     int64         
 2   month      10 non-null     int64         
 3   day        10 non-null     int64         
 4   hour       10 non-null     int64         
 5   minute     10 non-null     int64         
 6   status     10 non-null     object        
 7   to         10 non-null     object        
 8   from       8 non-null      object        
 9   latitude   10 non-null     float64       
 10  longitude  10 non-null     float64       
 11  port       10 non-null     object        
 12  speed      10 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(5), object(5)
memory usage: 1.1+ KB
None


<ipython-input-14-b697e4300f9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = pd.to_numeric(df['latitude'],errors = 'coerce')
<ipython-input-14-b697e4300f9a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = pd.to_numeric(df['longitude'],errors = 'coerce')


In [15]:
position = df[['latitude','longitude']]
position

,latitude,longitude
0,51.94967,3.98981
1,51.94974,3.99019
2,51.94987,3.99076
3,51.94982,3.99148
4,51.96252,4.07971
5,51.94307,3.43059
6,51.90014,3.20315
7,51.82419,2.89434
8,51.77846,2.70759
9,51.50706,2.27365


# Converting the position df to numpy array

In [16]:
numpy_position = position.to_numpy()
numpy_position 

array([[51.94967,  3.98981],
       [51.94974,  3.99019],
       [51.94987,  3.99076],
       [51.94982,  3.99148],
       [51.96252,  4.07971],
       [51.94307,  3.43059],
       [51.90014,  3.20315],
       [51.82419,  2.89434],
       [51.77846,  2.70759],
       [51.50706,  2.27365]])

# Sample code to choose the right Tile Layer for the map

In [17]:
#code to choose TileLayer

fig=Figure(width=550,height=350)
m2=folium.Map(location=[51.944800, 3.98981])
fig.add_child(m2)
folium.TileLayer('Stamen Terrain').add_to(m2)
folium.TileLayer('Stamen Toner').add_to(m2)
folium.TileLayer('Stamen Water Color').add_to(m2)
folium.TileLayer('cartodbpositron').add_to(m2)
folium.TileLayer('cartodbdark_matter').add_to(m2)
folium.LayerControl().add_to(m2)
m2

# First Map showing the ship route in blue

In [30]:

fig1=Figure(height=550,width=350)
m1=folium.Map(location=[51.944800, 3.98981],tiles='Stamen Water Color',zoom_start=11)
#fig5.add_child(m5)


# Creating feature groups
f1=folium.FeatureGroup("numpy_position")

# Adding lines to the different feature groups
line_1=folium.vector_layers.PolyLine(numpy_position,popup='<b>Path of ship</b>',tooltip='ship',color='blue',weight=10).add_to(f1)
f1.add_to(m1)

folium.LayerControl().add_to(m1)
m1

# Map 2 with Pins (markers)

In [31]:
fig2=Figure(width=550,height=350)
m2=folium.Map(location=[51.94967, 3.98981],tiles='Stamen Water Color',zoom_start=11)
fig2.add_child(m2)

#Adding markers to the map
folium.Marker(location=[51.94967,  3.98981],popup='Dock Marker1',tooltip='Click here to see Popup').add_to(m2)
folium.Marker(location=[51.94974,  3.99019],popup='<h3 style="color:green;">Marker2</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.94987,  3.99076],popup='<h3 style="color:green;">Marker3</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.94982,  3.99148],popup='<h3 style="color:green;">Marker4</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.96252,  4.07971],popup='<h3 style="color:green;">Marker5</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.94307,  3.43059],popup='<h3 style="color:green;">Marker6</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.90014,  3.20315],popup='<h3 style="color:green;">Marker7</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.82419,  2.89434],popup='<h3 style="color:green;">Marker8</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.77846,  2.70759],popup='<h3 style="color:green;">Marker9</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
folium.Marker(location=[51.50706,  2.27365],popup='<h3 style="color:green;">Marker10</h3>',tooltip='<strong>Click here to see Popup</strong>').add_to(m2)
m2

In [28]:
#pip install selenium
#import io
#from PIL import Image
#from selenium import webdriver
#browser = webdriver.Firefox()

#img_data = m3._to_png(5)
#img3 = Image.open(io.BytesIO(img_data))
#img3.save('image3_marker_pins.png')